In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#  import library

In [ ]:
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D , LSTM
from tensorflow.keras.optimizers import Adam
from keras import optimizers, callbacks 
from sklearn.metrics import log_loss
from keras.layers import Flatten
import matplotlib.pyplot as plt



In [ ]:
train_df = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
train_df.head(5)

In [ ]:
print(train_df.columns)
print(train_df['target'].value_counts())

In [ ]:
train_df['question_text'].str.len().hist()

In [ ]:
train_df['question_text'].str.split().map(lambda x : len(x)).hist(bins=64)


In [ ]:
max_length = 45

all most questions with max word is 45

In [ ]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and "}

rare word 

In [ ]:
all_text = ' '.join(train_df['question_text'])
all_text = all_text.split()
frequence  = pd.Series(all_text).value_counts()

one_word = frequence[frequence.values == 1]
one_word[5:20]

In [ ]:
def clean_question(x):
    if type(x) is str:
        x = x.lower() # transformer to lower 
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
            
        x = re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', '', x) # regex to remove to emails
       
        x = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x)   #regex to remove URLs     
        x = re.sub( u"\s+", u" ", x ).strip() # remove multiple  espace and back line
        x = ' '.join([t for t in x.split() if t not in one_word])  #combining all the text excluding rare words.
        return x
    else:
        return x

train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_question(x))        
train_df['question_text'] = train_df['question_text'].tolist()


In [ ]:
print(train_df['question_text'][:6])

In [ ]:

token = Tokenizer() 
token.fit_on_texts(train_df['question_text'])
vac = token.index_word


In [ ]:
dict(list(vac.items())[0:10]) 

In [ ]:
#declaring the vocab_size
vocab_size  = len(token.word_index) + 1
vocab_size

In [ ]:
##### conversion to numerical formats

encoded_text = token.texts_to_sequences(train_df['question_text'])
print(encoded_text[:3])

#padding='post' means that we padding post the sentence(keeping values 0 if the tokens are not there)

X = pad_sequences(encoded_text, maxlen=max_length, padding='post')
print(X[:2])

In [ ]:

glove_vectors = dict()

file = open('../input/nlpword2vecembeddingspretrained/glove.6B.200d.txt', encoding='utf-8')

for line in file:
    values = line.split()
    word = values[0]
    #storing the word in the variable
    vectors = np.asarray(values[1: ])
    #storing the vector representation of the respective word in the dictionary
    glove_vectors[word] = vectors
file.close()

#printing length of glove vectors
len(glove_vectors)


In [ ]:
glove_vectors.get('you').shape

Now we are creating a matrix for the tokens which we are having in our dataset and then storing their vector representation values in the matrix if it matches with glove_vectors words

In [ ]:
word_vector_matrix = np.zeros((vocab_size, 200))

for word, index in token.word_index.items():
    vector = glove_vectors.get(word)
    if vector is not None:
        word_vector_matrix[index] = vector

In [ ]:
word_vector_matrix[3]

In [ ]:
y = train_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)


In [ ]:
vector_size = 200
WEIGHTS_PATH = './w0.h5'
mc = callbacks.ModelCheckpoint( filepath=WEIGHTS_PATH, monitor='val_loss', mode='min', save_best_only=True )
model = Sequential()
model.add(Embedding(vocab_size, vector_size, input_length=max_length, weights = [word_vector_matrix], trainable = False))
model.add(LSTM(64))
model.add(Dense(100, activation='relu'))


model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate = 0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])
es = callbacks.EarlyStopping( patience=1 )
history = model.fit(X_train, y_train, epochs = 2, validation_data = (X_test, y_test) , callbacks=[es , mc] , batch_size=2048 )
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.legend( ['test', 'train'] )

In [ ]:
print(y_test[:10])

In [ ]:
preds = model.predict( X_test , batch_size=2048)

print ( 'test ssscore : ', preds[:10] ,log_loss(y_test, preds, eps = 1e-7) )